In [1]:
"""Sets the notebook to the project root"""

import sys, os
PROJECT_NAME = "analysis"
current_directory = os.getcwd().split("/")
while current_directory[-1] != PROJECT_NAME:
    current_directory = current_directory[:-1]
if len(current_directory) > 1:
    project_root = "/".join(current_directory)
    os.chdir(project_root)
else:
    raise Exception("ERROR: Project root not found")

from dotenv import load_dotenv
load_dotenv(dotenv_path="./env/env.env")

True

In [6]:
"""Initializes the database connection"""

import os
from bson import ObjectId
from src.classes.mongo_client import initialize

# Get MongoDB connection strings from environment variables
connection_strings = {
    "stg": os.getenv("MONGO_STAGE_URI"),
    "prd": os.getenv("MONGO_PROD_URI")
}

# Initialize database connections
dbs = initialize(connection_strings)



Database connections initialized successfully!
Staging DB: MongoClient(host=['landmarksid-staging-shard-00-02-me4vi.mongodb.net:27017', 'landmarksid-staging-shard-00-01-me4vi.mongodb.net:27017', 'landmarksid-staging-shard-00-00-me4vi.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='landmarksid-staging-shard-0', tls=True)
Production DB: MongoClient(host=['production-shard-00-01.gw4dh.mongodb.net:27017', 'production-shard-00-02.gw4dh.mongodb.net:27017', 'production-shard-00-00.gw4dh.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-4v0y1q-shard-0', tls=True)


In [7]:


sitecards = list(dbs["stg"]["poinetwork"]["sitecards"].find({"metadata.media_owner":"ONETProposal"}))

pois = []
for sitecard in sitecards:
    if len(sitecard["poi_candidates"]) > 0:
        pois.append(dbs["stg"]["poinetwork"]["pois"].find_one(
            {"_id": ObjectId(sitecard["poi_candidates"][0]["poiId"])}
        ))

print(len(pois))


111


In [ ]:

from pylandmarks.mongo import database, collections as mongo_collections, documents

clientNames = [
    "oOh Media", 
    "MediaWorks", 
    "JCDecaux", 
    "LUMO", 
    "VAST Billboards", 
    "Go Media - sales", 
    "Ad-Vantage Media", # ONET
    "Media5" # ONET
]

from src.classes.organizations import Organizations

org_data = Organizations()
media_owners = org_data.supportedOrganizations["MediaOwners"]


analysis = {}
for client in clientNames:
    analysis[client] = {}
    for o in [o for o in mongo_collections.organizations.supportedOrganizations["MediaOwners"] if o["archived"] == False]:
        if o["clientName"] == client:
            analysis[client][o["organizationName"]] = {
                "matchedPois": [],
                "totalPois": 0
            }


for sitecard in list(dbs["stg"]["poinetwork"]["sitecards"].find()):
    if len(sitecard["poi_candidates"]) > 0:
        poi = dbs["stg"]["poinetwork"]["pois"].find_one(
            {"_id": ObjectId(sitecard["poi_candidates"][0]["poiId"])}
        )

        for client in analysis:
            if poi["organizationName"] in analysis[client]:
                analysis[client][poi["organizationName"]]["matchedPois"].append(poi["_id"])

for client in analysis:
    for orgName in analysis[client]:
        analysis[client][orgName]["totalPois"] = len(list(dbs["stg"]["poinetwork"]["pois"].find({"organizationName": orgName})))
    
for client in analysis:
    for orgName in analysis[client]:
        print(f"{orgName} Total Pois: {analysis[client][orgName]["totalPois"]}, Matched Pois: {len(analysis[client][orgName]["matchedPois"])}, {round(len(analysis[client][orgName]["matchedPois"]) / analysis[client][orgName]["totalPois"] * 100, 2)}%")

ModuleNotFoundError: No module named 'pylandmarks'